<a href="https://colab.research.google.com/github/sergioopereira/AI/blob/main/mod4_redes_neurais/Code/NovoDataPrep/01_TemperaturaVerificada.v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import numpy as np
import pandas as pd
from datetime import datetime,date,time
from dateutil.relativedelta import relativedelta


### **A) Coleta dos Dados de Direção do Vento**

In [46]:
source_file = 'raw/BAUET2_CAETITÉ_2_TEM_Verif_EPE.txt'

In [47]:
df_temperatura = pd.read_csv(source_file, delimiter = ';' , header=None )

### **B) Sobre os Dados no Patio 1**

#### <span style="color:#DC143C">**B1.Formato dos Dados**

In [48]:
df_temperatura.shape # formato da matriz

(1622, 49)

#### <span style="color:#DC143C">**B2.Visao do Dataframe**

In [49]:
df_temperatura.head(2)  # visão de 5 linhas

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,20170101,999.0,999.000,18.733,18.300,17.867,17.4,17.000,16.833,16.633,16.333,16.067,15.900,15.767,15.833,16.267,16.9,17.633,18.233,18.733,19.367,20.100,20.767,21.400,22.100,22.733,23.433,24.033,24.800,25.033,25.567,25.900,26.067,26.633,26.700,26.500,26.133,25.967,25.667,25.367,25.467,25.733,25.700,25.767,25.667,25.5,23.8,22.100,21.833
1,20170102,21.3,20.667,20.267,19.833,19.000,18.6,18.333,17.900,17.567,17.400,17.233,17.133,17.133,17.167,17.300,17.7,17.967,19.267,20.600,21.467,22.333,23.133,23.733,24.467,25.000,25.767,26.200,26.967,27.567,27.800,27.867,28.500,28.567,28.667,29.367,28.467,27.800,27.333,27.267,27.267,27.300,27.267,27.233,26.533,26.1,25.0,22.933,22.900


### **C) Extração os Dados de 2017**

#### <span style="color:#DC143C"> **C1. Converter a primeira coluna em Data**

In [50]:
df_temperatura[0]= pd.to_datetime(df_temperatura[0], format='%Y%m%d')

#### <span style="color:#DC143C"> **C2.Remove registros fora do Intervalo**

In [51]:
df_temperatura = df_temperatura.drop(df_temperatura[df_temperatura[0] > '2017-12-31'].index)
df_temperatura = df_temperatura.drop(df_temperatura[df_temperatura[0] < '2017-01-01'].index)

In [52]:
df_temperatura['longitude'] = -42.75
df_temperatura['latitude']  = -14.25 

In [53]:
cols = list(df_temperatura)
cols = [cols[-1]] + cols[:-1]
df_temperatura = df_temperatura[cols]

cols = list(df_temperatura)
cols = [cols[-1]] + cols[:-1]
df_temperatura = df_temperatura[cols]

In [54]:
df_temperatura = df_temperatura.reset_index()

### **D) Formatação dos Dados**

#### <span style="color:#DC143C"> **D1. Migrar os dados para o Formato**

In [55]:
frente = ['Longitude', 'Latitude', 'Data', 'Valor30_1', 'Valor30_2']  # Cabeçalho
df_result_temperatura = pd.DataFrame(columns=frente)  # Criação de um dataframe vazio
    
Linha1    =  df_temperatura.shape[0] 
Colunas1  =  df_temperatura.shape[1] 

for lin2 in range(Linha1):  
    refhora = 0
    
    for col in range (Colunas1):        
        if col > 0:            
        
            if col % 2 != 0: # Se o número (col) for impar                   
               
                if col < 49:
                    
                    Data = df_temperatura.loc[lin2,0]
                                  
                    hora = time(hour=refhora, minute=0, second=0)            
                    dataCompleta = datetime.combine(Data, hora)
                   
                    lin_alvo = [df_temperatura.loc[lin2,'longitude'], df_temperatura.loc[lin2,'latitude'], dataCompleta , df_temperatura.loc[lin2,col], df_temperatura.loc[lin2,col+1]] 
                  
                    df_result_temperatura.loc[len(df_result_temperatura)] = lin_alvo  # adding a row
                         
                refhora = refhora + 1          
                


#### <span style="color:#DC143C"> **D2. Informações sobre os Tipos de Dados**

In [56]:
df_result_temperatura.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8760 entries, 0 to 8759
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Longitude  8760 non-null   float64       
 1   Latitude   8760 non-null   float64       
 2   Data       8760 non-null   datetime64[ns]
 3   Valor30_1  8760 non-null   float64       
 4   Valor30_2  8760 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 410.6 KB


In [57]:
df_result_temperatura.shape

(8760, 5)

#### <span style="color:#DC143C"> **D3. Remover ruido do dado**

In [58]:
df_result_temperatura.loc[df_result_temperatura['Valor30_1']== 999.00,'Valor30_1'] = 0
df_result_temperatura.loc[df_result_temperatura['Valor30_1']== 99.000,'Valor30_1'] = 0

df_result_temperatura.loc[df_result_temperatura['Valor30_2']== 99.000,'Valor30_2'] = 0
df_result_temperatura.loc[df_result_temperatura['Valor30_2']== 999.00,'Valor30_2'] = 0

save_file = 'd:\df_result_temperatura.csv'
df_result_temperatura.to_csv(save_file, index = True)

colMean1 = df_result_temperatura['Valor30_1'].sum() / ((df_result_temperatura['Valor30_1'] != 0).sum())
colMean1 = round(colMean1, 2)
print('Media 1:' + str(colMean1))

colMean2 = df_result_temperatura['Valor30_2'].sum() / ((df_result_temperatura['Valor30_2'] != 0).sum())
colMean2 = round(colMean2, 2)
print('Media 2:' + str(colMean2))

In [59]:
#df_result_temperatura.loc[df_result_temperatura['Valor30_1']== 0,'Valor30_1'] = 19.53
#df_result_temperatura.loc[df_result_temperatura['Valor30_2']== 0,'Valor30_2'] = 19.54

#df_result_temperatura.loc[df_result_temperatura['Valor30_1']>32 ,'Valor30_1'] = 19.53
#df_result_temperatura.loc[df_result_temperatura['Valor30_2']>32 ,'Valor30_2'] = 19.54

In [60]:
# Removendo as linhas sem valores
df_result_temperatura = df_result_temperatura.drop(df_result_temperatura[df_result_temperatura['Valor30_1'] == 0].index)
df_result_temperatura = df_result_temperatura.drop(df_result_temperatura[df_result_temperatura['Valor30_2'] == 0].index)
df_result_temperatura = df_result_temperatura.drop(df_result_temperatura[df_result_temperatura['Valor30_1'] > 32].index)
df_result_temperatura = df_result_temperatura.drop(df_result_temperatura[df_result_temperatura['Valor30_2'] > 32].index)

#### <span style="color:#DC143C"> **D3. Agregar Magnitude para 1H**

In [61]:
df_result_temperatura['Temperatura_Verif'] = (df_result_temperatura['Valor30_1'] + df_result_temperatura['Valor30_2']) / 2

In [62]:
# Formatar o resultado com 2 casas decimais
df_result_temperatura['Temperatura_Verif'] = df_result_temperatura['Temperatura_Verif'].round(decimals=2)

In [63]:
df_result_temperatura = df_result_temperatura.drop('Valor30_1', 1)
df_result_temperatura = df_result_temperatura.drop('Valor30_2', 1)

In [64]:
df_result_temperatura.head(-2)

,Longitude,Latitude,Data,Temperatura_Verif
1,-42.75,-14.25,2017-01-01 01:00:00,18.52
2,-42.75,-14.25,2017-01-01 02:00:00,17.63
3,-42.75,-14.25,2017-01-01 03:00:00,16.92
4,-42.75,-14.25,2017-01-01 04:00:00,16.48
5,-42.75,-14.25,2017-01-01 05:00:00,15.98
...,...,...,...,...
8752,-42.75,-14.25,2017-12-31 16:00:00,26.87
8753,-42.75,-14.25,2017-12-31 17:00:00,26.22
8754,-42.75,-14.25,2017-12-31 18:00:00,25.28
8755,-42.75,-14.25,2017-12-31 19:00:00,24.88


### **E. Exporta Resultados**

In [65]:
output_file = 'processed/teste_03_patio_temperatura.csv'
df_result_temperatura.to_csv(output_file, sep=';', encoding='utf-8')